**Packages**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from datasets import load_dataset
import evaluate

#let's make longer output readable without horizontal scrolling
from pprint import pprint

import warnings

import regex as re

/home/ubuntu/w266/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-01 05:50:12.542574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-01 05:50:13.397016: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/C

**Necessary Functions**

In [2]:
import inspect

def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

In [3]:
rouge = evaluate.load('rouge')

In [4]:
chrf = evaluate.load("chrf")

**Data**

In [5]:
dataset = load_dataset("csebuetnlp/xlsum", "english")

Found cached dataset xlsum (/home/ubuntu/.cache/huggingface/datasets/csebuetnlp___xlsum/english/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 324.73it/s]


In [6]:
index = pd.DataFrame({"index": list(range(len(dataset['train'])))})
sample_index = index.sample(n=2000, replace=False, random_state=1004)
sample_index[:5]

,index
235420,235420
172024,172024
253546,253546
224954,224954
214134,214134


In [7]:
id = []
url = []
title = []
article = []
article_num_sentences = []
article_num_characters = []
summary = []
summary_num_sentences = []
summary_num_characters = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    summary_num_sentences.append(len(dataset["train"][i]['summary'].split(".")))
    summary_num_characters.append(len(dataset["train"][i]['summary']))
    article.append(dataset["train"][i]['text'])
    article_num_sentences.append(len(dataset["train"][i]['text'].split(".")))
    article_num_characters.append(len(dataset["train"][i]['text']))

KeyboardInterrupt: 

In [ ]:
d = {'id': id, 'url': url, "title": title, 'article': article, "article_num_sentences": article_num_sentences, "article_num_characters": article_num_characters, 'summary': summary,"summary_num_sentences": summary_num_sentences, "summary_num_characters": summary_num_characters}
df = pd.DataFrame(data=d)
df.head(5)

**Default Hyperparameters**

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer()

In [ ]:
get_default_args(Summarizer)

In [ ]:
get_default_args(model)

**BERT Extractive Model**

Model 0: Default Hyperparameters

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer()

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []
bert_chrf = []

for i in range(int(len(df['article'])/2)):
    
    candidate = model(df['article'][i],
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    bert_chrf.append(results2['score'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_0_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_0_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

Finetuning On First 1000

Model 1. Adjusted min_length, max_length, num_sentences (which overrides ratio)

In [ ]:
from summarizer import Summarizer

In [ ]:
# Using bert-base instead of bert-large to reduce run times
model = Summarizer(model='bert-base-uncased')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    bert_chrf.append(results2['score'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_1_scores.csv', index=False)
        print(i)
        
data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_1_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

Tuning Hyperparameters

2. Made num_sentences based off number of clusters instead of average 

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased',
                  )

In [ ]:
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):

    res = model.calculate_optimal_k(df['article'][i], k_max=10)
    
    candidate = model(df['article'][i], 
                      num_sentences = res, # number of sentences determined by number of clusters
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,)
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_2_scores.csv', index=False)
        print(i)
        
data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_2_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

3. Setting number of sentences with clusters didn't help in second model, so we are reverting back to first model but setting use_first = False bc many examples have meta data in first sentence.

In [ ]:
from summarizer import Summarizer

warnings.filterwarnings("ignore")

model = Summarizer(model='bert-base-uncased',
                  )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = False,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref
    
    bert_chrf.append(results2['score'])
                            
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_3_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_3_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

4. Setting use_first = True

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased',
                  )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = True,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_3_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_3_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

5. Use bert-large-uncased w/ best from above

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-large-uncased',
                  )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = False,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_4_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_4_scores.csv', index=False)
print(i)

In [ ]:
# this model took longer and performed worse
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

6. Reduce option median

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased', reduce_option = 'median'
                  )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_4_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_6_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

**7. Reduce option max THIS WORKED THIS BEST!!!**

In [ ]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased', reduce_option = 'max'
                  )

In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores.csv', index=False)
print(i)

In [ ]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))
print('chrf average:', np.mean(bert_chrf))

In [ ]:
print('num_sentences', round(df["summary_num_sentences"][:1000].mean()) 
print('min_length', min(df["summary_num_characters"][:1000])
print('max_length', max(df["summary_num_characters"][:1000])

**Best BERT Extractive model from finetuning above by category**

In [ ]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

In [ ]:
categories = []

for i in range(len(dataset['train'])):
    cat = dataset['train'][i]['id']
    result = re.sub('\d','',cat)[:-1]
    result = result.split('-')[0].split('.')[0]
    categories.append(result)

**Category 1: uk**

In [26]:
uk = find_indices(categories, 'uk')
index = pd.DataFrame({"index": uk})
sample_index = index.sample(n=1000, replace=False, random_state=1004)
sample_index[:5]

,index
103144,184549
135979,242139
122918,219346
41576,76571
23230,44271


In [27]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [28]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,summary
0,uk-england-bristol-51032231,https://www.bbc.com/news/uk-england-bristol-51...,Author Emily Koch: 'I'm not angry at the drive...,Emily Koch suffered two broken legs and ligame...,A woman left seriously injured after being hit...
1,uk-england-birmingham-54264699,https://www.bbc.com/news/uk-england-birmingham...,Blind TikTok star Lucy Edwards says reaction t...,"Lucy Edwards, who became Radio 1's first blind...",A blind vlogger hopes her TikTok videos on liv...
2,uk-england-london-16091997,https://www.bbc.com/news/uk-england-london-160...,Bendy bus makes final journey for Transport fo...,The vehicles were used on 12 routes over the p...,The last of London's bendy buses was taken off...
3,uk-england-norfolk-54058083,https://www.bbc.com/news/uk-england-norfolk-54...,Horsey seals: Volunteers remove rubber ring fr...,Four members of Friends of Horsey Seals netted...,"Volunteers have helped capture a ""feisty"" seal..."
4,uk-scotland-edinburgh-east-fife-28838287,https://www.bbc.com/news/uk-scotland-edinburgh...,Tim Vine wins funniest Edinburgh Fringe joke a...,"He won with the one-liner: ""I decided to sell ...",A joke by comedian Tim Vine about a vacuum cle...


In [29]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores_uk.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores_uk.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [30]:
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))
print('chrf average:', np.mean(base_chrf))

rouge1 average:  0.18939253006007692
rouge2 average:  0.027133586978308793
rougeL average:  0.12278381086366534
rougeLs average: 0.12278381086366534
chrf average: 26.572484469570067


**Category 2: world**

In [35]:
cat_inx = find_indices(categories, 'world')
index = pd.DataFrame({"index": cat_inx})
sample_index = index.sample(n=1000, replace=False, random_state=1004)
sample_index[:5]

,index
38266,209050
5910,25805
14047,71645
23996,127936
25699,137681


In [36]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [37]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,summary
0,world-asia-53847400,https://www.bbc.com/news/world-asia-53847400,Kim Jong-un gives sister Yo-jong 'more respons...,"Mr Kim still maintains ""absolute authority"", b...",North Korean leader Kim Jong-un has delegated ...
1,world-europe-guernsey-18129181,https://www.bbc.com/news/world-europe-guernsey...,Guernsey overgrown trees prompt road crash fears,Overgrown bushes and trees could result in peo...,Residents are being urged to trim vegetation o...
2,world-africa-44629681,https://www.bbc.co.uk/news/world-africa-44629681,Bringing Gay Pride to Africa's last absolute m...,And anyone doubting the determination needed t...,Africa's last absolute monarchy is holding its...
3,world-asia-41840069,https://www.bbc.com/news/world-asia-41840069,Pakistan polygamy: Lahore man jailed over unap...,The court in Lahore also ordered Shahzad Saqib...,A Pakistan court has jailed a man for six mont...
4,world-europe-guernsey-13052445,https://www.bbc.com/news/world-europe-guernsey...,Alternative recycling bank site proposed,The old facility at Manor Stores closed last y...,Work is continuing to try to find a permanent ...


In [38]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores_world.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores_world.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [39]:
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))
print('chrf average:', np.mean(base_chrf))

rouge1 average:  0.19131117325937153
rouge2 average:  0.026498254102528457
rougeL average:  0.12255157736810432
rougeLs average: 0.12255157736810432
chrf average: 27.301136246709188


**Category 3: business**

In [40]:
cat_inx = find_indices(categories, 'business')
index = pd.DataFrame({"index": cat_inx})
sample_index = index.sample(n=1000, replace=False, random_state=1004)
sample_index[:5]

,index
17751,250222
8987,132229
2855,46895
17482,246643
13624,194415


In [41]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [42]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,summary
0,business-51487191,https://www.bbc.co.uk/news/business-51487191,British Gas owner Centrica hit by energy price...,Centrica made a loss of £849m in the year ende...,British Gas owner Centrica has blamed a big lo...
1,business-18289184,https://www.bbc.com/news/business-18289184,Pakistan economy at a crossroads,By Shahzeb JillaniBBC News The good news is th...,Pakistan is a nuclear weapon state with a popu...
2,business-48524850,https://www.bbc.com/news/business-48524850,Green secures rescue deal for Topshop empire,"The plan, covering stores including Topshop, M...",Sir Philip Green's Arcadia retail empire has b...
3,business-54004709,https://www.bbc.com/news/business-54004709,"Coronavirus: 'I took a £1,000 flight to beat P...",By Simon ReadBusiness reporter But he had to s...,John Cushing is cutting his Portugal holiday s...
4,business-37339981,https://www.bbc.com/news/business-37339981,Tax credits mum 'falsely accused of marriage t...,By Peter Whittlesea & John OwenVictoria Derbys...,A teenage mother had her child tax credits sto...


In [43]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores_business.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores_business.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [44]:
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))
print('chrf average:', np.mean(base_chrf))

rouge1 average:  0.1873668697599981
rouge2 average:  0.026998209932834666
rougeL average:  0.12229664365938817
rougeLs average: 0.12229664365938817
chrf average: 26.8081030850095


**Category 4: entertainment**

In [45]:
cat_inx = find_indices(categories, 'entertainment')
index = pd.DataFrame({"index": cat_inx})
sample_index = index.sample(n=1000, replace=False, random_state=1004)
sample_index[:5]

,index
5668,127955
7275,163834
4655,106634
12025,266862
2458,58192


In [46]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [47]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,summary
0,entertainment-arts-19783861,https://www.bbc.com/news/entertainment-arts-19...,National Theatre reports record annual takings...,By Helen BushbyBBC Entertainment and Arts repo...,The National Theatre has reported record yearl...
1,entertainment-arts-38290366,https://www.bbc.com/news/entertainment-arts-38...,Golden Globes: La La Land leads nominations,Damien Chazelle's movie is up for best musical...,Musical film La La Land leads the pack in this...
2,entertainment-arts-28228750,https://www.bbc.com/news/entertainment-arts-28...,Yorkshire Sculpture Park wins museum of the ye...,By Tim MastersArts and entertainment correspon...,The open-air Yorkshire Sculpture Park has been...
3,entertainment-arts-36979491,https://www.bbc.com/news/entertainment-arts-36...,New Alf Garnett seen in Till Death Us Do Part ...,"Simon Day, who has previously appeared in The ...",Alf Garnett has lost his signature moustache b...
4,entertainment-arts-28834671,https://www.bbc.com/news/entertainment-arts-28...,BSkyB makes 20% ethnic minorities pledge,The target applies to all home-grown programme...,"The broadcaster, Sky, has pledged that 20% of ..."


In [ ]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    bert_chrf.append(results2['score'])
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs, 'chrf': bert_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700


In [ ]:
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))
print('chrf average:', np.mean(base_chrf))

**Category 5: technology**

In [ ]:
cat_inx = find_indices(categories, 'technology')
index = pd.DataFrame({"index": cat_inx})
sample_index = index.sample(n=1000, replace=False, random_state=1004)
sample_index[:5]

In [ ]:
id = []
url = []
title = []
article = []
summary = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    article.append(dataset["train"][i]['text'])

In [ ]:
d = {'id': id, 'url': url, "title": title, 'article': article, 'summary': summary}
df = pd.DataFrame(data=d)
df.head(5)

In [ ]:
base_r1 = []
base_r2 = []
base_rL = []
base_rLs = []
base_chrf = []

for i in range(len(df['article'])): 
    
    
    # first three sentences 
    candidate = ". ".join(df["article"][i].split('. ')[0:3]) + "."
    candidate = [candidate]
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references= ref)
    
    results2 = chrf.compute(predictions=candidate,
                            references= ref)
    
    base_r1.append(results['rouge1'])
    base_r2.append(results['rouge2'])
    base_rL.append(results['rougeL'])
    base_rLs.append(results['rougeLsum'])
    
    base_chrf.append(results2['score'])
    
    if i in np.arange(0, 2000, 100):
        data = {'rouge1': base_r1, 'rouge2': base_r2, 'rogueL': base_rL, 'rogueLs': base_rLs, 'chrf': base_chrf}
        scores = pd.DataFrame(data)
        scores.to_csv(r'base_scores_tech.csv', index=False)
        print(i)
        
data = {'rouge1': base_r1, 'rouge2': base_r2, 'rogueL': base_rL, 'rogueLs': base_rLs, 'chrf': base_chrf}
scores = pd.DataFrame(data)
scores.to_csv(r'base_scores_tech.csv', index=False)
print(i)

In [ ]:
print('rouge1 average: ', np.mean(base_r1))
print('rouge2 average: ', np.mean(base_r2))
print('rougeL average: ', np.mean(base_rL))
print('rougeLs average:', np.mean(base_rLs))
print('chrf average:', np.mean(base_chrf))